In [ ]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes
!pip install datasets sentencepiece

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-8kzouw17/unsloth_c39c9e2e80994974a810763220a6df73
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-8kzouw17/unsloth_c39c9e2e80994974a810763220a6df73
  Resolved https://github.com/unslothai/unsloth.git to commit 7a8f99e1890213cdd01a3ab6c3e13174a96e8220
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 110.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.3/124.3 kB 12.3 MB/s eta 0:00:

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.5/31.5 MB 71.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 17.0 MB/s eta 0:00:00
  Attempting uninstall: trl
    Found existing installation: trl 0.15.2
    Uninstalling trl-0.15.2:
      Successfully uninstalled trl-0.15.2


In [ ]:
import torch
import os
import gc
import time
import re
from typing import List, Literal, Optional


from unsloth import PatchDPOTrainer
PatchDPOTrainer()
print("Applied Unsloth PatchDPOTrainer.")

from unsloth import FastLanguageModel, is_bfloat16_supported
from trl import DPOTrainer
from transformers import TrainingArguments
from datasets import load_dataset, DatasetDict, concatenate_datasets
from datasets.builder import DatasetGenerationError

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: Failed to patch SmolVLMForConditionalGeneration forward function.


    PyTorch 2.7.0+cu126 with CUDA 1206 (you have 2.6.0+cu124)
    Python  3.11.12 (you have 3.11.12)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


🦥 Unsloth Zoo will now patch everything to make training faster!
Applied Unsloth PatchDPOTrainer.


In [ ]:
model_name = "unsloth/Phi-3-mini-4k-instruct-bnb-4bit"
max_seq_length = 1024
dtype = None
load_in_4bit = True
output_directory_dpo = "phi3_mini_dpo_unslothdoc_run1"
dataset_sample_percent = 0.005
dpo_learning_rate = 5e-6
dpo_num_epochs = 3
dpo_beta = 0.1

In [ ]:
start_time = time.time()
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)
end_time = time.time()
print(f"Model loaded in {end_time - start_time:.2f}s.")

if tokenizer.chat_template is None:
    print("WARNING: tokenizer.chat_template is None. Manually setting Phi-3 template.")
    phi3_template = """{% set loop_messages = messages %}{% for message in loop_messages %}{% set content = '<|' + message['role'] + '|>\n' + message['content'] | trim + '<|end|>\n' %}{% if loop.index0 == 0 %}{% set content = bos_token + content %}{% endif %}{{ content }}{% endfor %}{% if add_generation_prompt %}{{ '<|assistant|>\n' }}{% endif %}"""
    tokenizer.chat_template = phi3_template
    print("Manually set Phi-3 chat template.")
else:
    print("tokenizer.chat_template is already set.")

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    print("Set pad_token = eos_token.")
if tokenizer.bos_token is None: print("Warning: BOS token not set in tokenizer.")
if tokenizer.eos_token is None: print("Warning: EOS token not set in tokenizer.")

==((====))==  Unsloth 2025.4.1: Fast Mistral patching. Transformers: 4.51.3.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.26G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/194 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/3.34k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/458 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

Model loaded in 23.67s.
tokenizer.chat_template is already set.


In [ ]:
def apply_chat_template(
    example,
    tokenizer,
    task: Literal["sft", "generation", "rm", "dpo"] = "sft",
    assistant_prefix="<|assistant|>\n",
):
    def _strip_prefix(s, pattern):
        if not s: return s
        return re.sub(f"^{re.escape(pattern)}", "", s)

    if task == "dpo":
        if all(k in example.keys() for k in ("chosen", "rejected")):
            # Check if chosen/rejected are lists and non-empty
            if not isinstance(example["chosen"], list) or not example["chosen"]:
                 print("Warning: 'chosen' field is not a non-empty list.")
                 return {"text_prompt": "", "text_chosen": "", "text_rejected": ""}
            if not isinstance(example["rejected"], list) or not example["rejected"]:
                 print("Warning: 'rejected' field is not a non-empty list.")
                 return {"text_prompt": "", "text_chosen": "", "text_rejected": ""}

            # Ensure messages are dicts with 'role' and 'content'
            if not all(isinstance(msg, dict) and 'role' in msg and 'content' in msg for msg in example["chosen"]):
                 print("Warning: 'chosen' messages have incorrect format.")
                 return {"text_prompt": "", "text_chosen": "", "text_rejected": ""}
            if not all(isinstance(msg, dict) and 'role' in msg and 'content' in msg for msg in example["rejected"]):
                 print("Warning: 'rejected' messages have incorrect format.")
                 return {"text_prompt": "", "text_chosen": "", "text_rejected": ""}

            # Extract prompt from the first user message
            prompt_messages = []
            system_message = None
            if example["chosen"][0]["role"] == "system":
                 system_message = example["chosen"][0]
                 user_prompt_msg = next((msg for msg in example["chosen"] if msg["role"] == "user"), None)
            else:
                 user_prompt_msg = next((msg for msg in example["chosen"] if msg["role"] == "user"), None)

            if system_message: prompt_messages.append(system_message)
            if user_prompt_msg: prompt_messages.append(user_prompt_msg)

            if not user_prompt_msg:
                print("Warning: Could not find user prompt in 'chosen' messages.")
                prompt_messages = [system_message] if system_message else []


            chosen_assistant_msgs = [msg for msg in example["chosen"] if msg["role"] == "assistant"]
            rejected_assistant_msgs = [msg for msg in example["rejected"] if msg["role"] == "assistant"]

            example["text_prompt"] = tokenizer.apply_chat_template(
                prompt_messages, tokenize=False, add_generation_prompt=True
            )

            chosen_response_str = "\n".join([msg["content"] for msg in chosen_assistant_msgs]).strip()
            rejected_response_str = "\n".join([msg["content"] for msg in rejected_assistant_msgs]).strip()

            example["text_chosen"] = chosen_response_str
            example["text_rejected"] = rejected_response_str

        else:
            raise ValueError(
                f"Could not format example as dialogue for `dpo` task! Require `[chosen, rejected]` keys but found {list(example.keys())}"
            )
    else:
         raise ValueError(f"Task {task} not supported by this adapted function.")

    if not prompt_messages:
        example["text_prompt"] = ""

    return example

In [ ]:
dataset_name = "HuggingFaceH4/ultrafeedback_binarized"
train_split = "train_prefs"
test_split = "test_prefs"

# Load datasets
try:
    raw_datasets = DatasetDict()
    raw_datasets["train"] = load_dataset(dataset_name, split=train_split)
    print(f"Loaded splits: {list(raw_datasets.keys())}")
except ValueError as e:
    print(f"Warning: Could not load splits '{train_split}'/'{test_split}'. Trying 'train'/'test'. ({e})")
    train_split = "train"
    test_split = "test"
    raw_datasets["train"] = load_dataset(dataset_name, split=train_split)
    print(f"Loaded splits: {list(raw_datasets.keys())}")

# Sample the training dataset
original_size = len(raw_datasets["train"])
sample_size = int(dataset_sample_percent * original_size)
raw_datasets["train"] = raw_datasets["train"].shuffle(seed=42).select(range(sample_size))
print(f"Sampled training set to {len(raw_datasets['train'])} examples ({dataset_sample_percent*100:.1f}%).")

# Store original column names (important for remove_columns)
original_columns = list(raw_datasets["train"].features)
print(f"Original columns: {original_columns}")

# Apply the formatting function
print("\nApplying chat template formatting (task='dpo')...")
num_proc = os.cpu_count() // 2 if os.cpu_count() else 1 # Use ~half CPU cores
print(f"Using num_proc={num_proc}")

# Wrap map call in try-except for better debugging
try:
     raw_datasets = raw_datasets.map(
         apply_chat_template,
         fn_kwargs={"tokenizer": tokenizer, "task": "dpo"},
         num_proc=num_proc,
         remove_columns=original_columns,
         desc="Formatting comparisons with prompt template",
     )
     print("Formatting map complete.")
except Exception as e:
     print(f"\n--- ERROR during dataset map ---")
     print(e)
     import traceback
     traceback.print_exc()
     print("------------------------------")
     raise

# Rename columns to what TRL DPOTrainer expects by default
print("\nRenaming columns...")
expected_new_cols = ["text_prompt", "text_chosen", "text_rejected"]
actual_cols = list(raw_datasets["train"].features)
if not all(col in actual_cols for col in expected_new_cols):
    print(f"*** ERROR: Expected columns {expected_new_cols} not found after mapping. Found: {actual_cols}. Check apply_chat_template function. ***")
    raise KeyError("Missing expected columns after formatting map.")

# Perform renaming for both train and test splits if they exist
for split in raw_datasets.keys():
    raw_datasets[split] = raw_datasets[split].rename_columns(
        {"text_prompt": "prompt", "text_chosen": "chosen", "text_rejected": "rejected"}
    )
    print(f"Renamed columns in '{split}' split.")

README.md:   0%|          | 0.00/6.53k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


train_prefs-00000-of-00001.parquet:   0%|          | 0.00/226M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


test_prefs-00000-of-00001.parquet:   0%|          | 0.00/7.29M [00:00<?, ?B/s]

...

Generating train_gen split:   0%|          | 0/61135 [00:00<?, ? examples/s]

Generating test_gen split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Loaded splits: ['train']
Sampled training set to 305 examples (0.5%).
Original columns: ['prompt', 'prompt_id', 'chosen', 'rejected', 'messages', 'score_chosen', 'score_rejected']

Applying chat template formatting (task='dpo')...
Using num_proc=6


Formatting comparisons with prompt template (num_proc=6):   0%|          | 0/305 [00:00<?, ? examples/s]

Formatting map complete.

Renaming columns...
Renamed columns in 'train' split.


In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 64,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 64,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
)

Unsloth 2025.4.1 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
gc.collect(); torch.cuda.empty_cache()

# Check dataset exists and has the train split
if "train" not in raw_datasets: raise KeyError("Missing 'train' split in prepared `raw_datasets`.")
if len(raw_datasets["train"]) == 0: raise ValueError("Training dataset is empty.")

try:
    dpo_trainer = DPOTrainer(
        model = model,
        ref_model = None,
        args = TrainingArguments(
            per_device_train_batch_size = 2,
            gradient_accumulation_steps = 4,
            warmup_ratio = 0.1,
            num_train_epochs = dpo_num_epochs,
            learning_rate = dpo_learning_rate,
            fp16 = not is_bfloat16_supported(),
            bf16 = is_bfloat16_supported(),
            logging_steps = 10,
            optim = "adamw_8bit",
            weight_decay = 0.0,
            lr_scheduler_type = "linear",
            seed = 42,
            output_dir = output_directory_dpo,
            save_strategy = "epoch",
            report_to = "none",
            remove_unused_columns = False
        ),
        beta = dpo_beta,
        train_dataset = raw_datasets["train"],
        tokenizer = tokenizer,
        max_length = max_seq_length,
        max_prompt_length = max_seq_length // 2,
        max_target_length = max_seq_length // 2,
    )
    print("DPOTrainer configured successfully using TrainingArguments.")

except Exception as e:
    print(f"Error configuring DPOTrainer: {e}")
    import traceback
    traceback.print_exc()
    raise

Trainer.tokenizer is now deprecated. You should use `Trainer.processing_class = processing_class` instead.


Map:   0%|          | 0/305 [00:00<?, ? examples/s]

Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Tr

DPOTrainer configured successfully using TrainingArguments.


In [ ]:
gc.collect()
if torch.cuda.is_available(): torch.cuda.empty_cache(); print("Cleared CUDA cache.")

start_train_time = time.time()
try:
    dpo_trainer.train()
    end_train_time = time.time()
    print(f"\nDPO Training finished in {(end_train_time - start_train_time)/60:.2f} minutes.")
    print("=== DPO Training Complete ===")
except Exception as e:
    print(f"\n--- ERROR DURING DPO TRAINING ---"); print(e)
    import traceback; traceback.print_exc(); print("---------------------------------")

final_adapter_dir_dpo = f"{output_directory_dpo}/final_adapters" # Save within output dir
print(f"\n--- Saving final DPO LoRA adapters to: {final_adapter_dir_dpo} ---")
try:
    dpo_trainer.model.save_pretrained(final_adapter_dir_dpo)
    tokenizer.save_pretrained(final_adapter_dir_dpo)
    print(f"DPO Adapters and tokenizer saved.")
    !ls -lh {final_adapter_dir_dpo}
except Exception as e: print(f"Error saving adapters: {e}")

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 305 | Num Epochs = 3 | Total steps = 114
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 119,537,664/4,000,000,000 (2.99% trained)


Cleared CUDA cache.

--- ERROR DURING DPO TRAINING ---
DPOTrainer.get_batch_samples() takes 3 positional arguments but 4 were given
---------------------------------

--- Saving final DPO LoRA adapters to: phi3_mini_dpo_unslothdoc_run1/final_adapters ---


Traceback (most recent call last):
  File "<ipython-input-9-3a64a69ea395>", line 6, in <cell line: 0>
    dpo_trainer.train()
  File "/usr/local/lib/python3.11/dist-packages/transformers/trainer.py", line 2245, in train
    return inner_training_loop(
           ^^^^^^^^^^^^^^^^^^^^
  File "<string>", line 268, in _fast_inner_training_loop
TypeError: DPOTrainer.get_batch_samples() takes 3 positional arguments but 4 were given


DPO Adapters and tokenizer saved.
total 461M
-rw-r--r-- 1 root root  869 Apr 29 01:10 adapter_config.json
-rw-r--r-- 1 root root 457M Apr 29 01:10 adapter_model.safetensors
-rw-r--r-- 1 root root  293 Apr 29 01:10 added_tokens.json
-rw-r--r-- 1 root root 5.0K Apr 29 01:10 README.md
-rw-r--r-- 1 root root  572 Apr 29 01:10 special_tokens_map.json
-rw-r--r-- 1 root root 3.3K Apr 29 01:10 tokenizer_config.json
-rw-r--r-- 1 root root 3.5M Apr 29 01:10 tokenizer.json
-rw-r--r-- 1 root root 489K Apr 29 01:10 tokenizer.model


In [ ]:
import warnings
warnings.filterwarnings("ignore")

try:
    inference_model = dpo_trainer.model
    FastLanguageModel.for_inference(inference_model)
    inference_model.eval()
    print("Using model from trainer for inference.")
except NameError: # Fallback if trainer object doesn't exist
    print("Reloading model and adapters for inference...")
    from peft import PeftModel
    model, tokenizer = FastLanguageModel.from_pretrained(model_name, max_seq_length=max_seq_length, dtype=dtype, load_in_4bit=load_in_4bit)
    if tokenizer.chat_template is None:
        phi3_template = """{% set loop_messages = messages %}{% for message in loop_messages %}{% set content = '<|' + message['role'] + '|>\n' + message['content'] | trim + '<|end|>\n' %}{% if loop.index0 == 0 %}{% set content = bos_token + content %}{% endif %}{{ content }}{% endfor %}{% if add_generation_prompt %}{{ '<|assistant|>\n' }}{% endif %}"""
        tokenizer.chat_template = phi3_template
    if tokenizer.pad_token is None: tokenizer.pad_token = tokenizer.eos_token
    inference_model = PeftModel.from_pretrained(model, final_adapter_dir_dpo)
    FastLanguageModel.for_inference(inference_model)
    inference_model.eval()
    print("Model reloaded.")

test_prompt = "How can I learn programming effectively?"
messages = [{"role": "user", "content": test_prompt}]
inputs = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt").to("cuda" if torch.cuda.is_available() else "cpu")
if not isinstance(inputs, torch.Tensor) and hasattr(inputs, 'input_ids'): inputs = inputs.input_ids

generation_params = { "max_new_tokens": 250, "use_cache": True, "do_sample": True, "temperature": 0.7, "top_p": 0.9, "eos_token_id": tokenizer.eos_token_id, "pad_token_id": tokenizer.eos_token_id, }

print("\nGenerating DPO model response...")
response = "[Generation Error]"
try:
    with torch.no_grad(): outputs = inference_model.generate(inputs, **generation_params)
    input_len, output_len = inputs.shape[-1], outputs.shape[-1]
    if output_len > input_len: response = tokenizer.decode(outputs[0][input_len:], skip_special_tokens=True).strip()
    else: response = "[No new tokens]"
except Exception as e: print(f"Generation/Decoding Error: {e}")

print(f"\nPrompt: {test_prompt}"); print(f"\nDPO Model Response:\n{response}")
del inference_model; del inputs; gc.collect(); torch.cuda.empty_cache()

Using model from trainer for inference.

Generating DPO model response...
Generation/Decoding Error: 'NoneType' object has no attribute 'attn_bias'

Prompt: How can I learn programming effectively?

DPO Model Response:
[Generation Error]
